In [97]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

In [98]:
import openai
openai.organization = organization
openai.api_key = api_key

In [99]:
gpt_model_name = "gpt-3.5-turbo"

In [100]:
def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message

In [101]:
basic_completion("Write a haiku about being a dungeon master")

<OpenAIObject at 0x7f21af2a9d50> JSON: {
  "content": "Crafting worlds of lore,\nDice roll fate, adventure calls,\nDungeon Master's realm.",
  "role": "assistant"
}

In [102]:
system_prompt = '''
You are a dungeon master assistant. You will create and improvise world content, behaviors, and events.

World lore and game mechanics should be compatible with dungeons and dragons standard fifth edition (5e).

All response MUST be in valid YAML format, and MUST NOT contain any other characters.
The user expects to be able to parse each raw response as YAML. The response should NOT be formatted as Markdown.
'''

import yaml

def system_completion(user_prompt):
    amended_user_prompt = f"""
    {user_prompt}

    The response MUST be in valid YAML format, and MUST NOT contain any other characters.
    The user expects to be able to parse each raw response as YAML. The response should NOT be formatted as Markdown.
    """

    call = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': amended_user_prompt}
        ],
    )
    raw_message = call.choices[0].message['content']
    parsed_message = yaml.safe_load(raw_message)
    return (raw_message, parsed_message)


In [103]:
(raw_name, parsed_name) = system_completion("Generate a name for a fantasy world")
print(raw_name)

world_name: Eldridia


In [132]:
def range(pair): return f"between {pair[0]} and {pair[1]}"

def geography_prompt(continents=(2,4), regions=(12,15)): return f'''
Generate the geography of a fictional fantasy world, that will be the setting for the game.

The world exists on a flat map whose coordinates range from -1000 to +1000 miles, both south-north and west-east, centered at the origin.

There must be {range(regions)} total regions in the response. There must be {range(continents)} continents in the response.

Regions should represent land regions. Regions on the same continent should be roughly contiguous, and regions on different continents should
not be adjacent. There should be plenty of space left on the map for large bodies of water, such as oceans and seas. The bounding boxes of regions
should mostly not overlap. The continents should be spread out across the world map.

The response must be in the following structure. The order of the generated fields matters.

continents:
    name: <string>
    regions:
        center_latitude: <int>
        center_longitude: <int>
        width_miles: <int>
        height_miles: <int>
        topography: <string>
        climate: <string>
        biome: <string>
        size: <string>
        name: <string>

IT IS VERY IMPORTANT that you keep going until enough regions and continents have been generated.
'''

print(geography_prompt())


Generate the geography of a fictional fantasy world, that will be the setting for the game.

The world exists on a flat map whose coordinates range from -1000 to +1000 miles, both south-north and west-east, centered at the origin.

There must be between 12 and 15 total regions in the response. There must be between 2 and 4 continents in the response.

Regions should represent land regions. Regions on the same continent should be roughly contiguous, and regions on different continents should
not be adjacent. There should be plenty of space left on the map for large bodies of water, such as oceans and seas.

The response must be in the following structure. The order of the generated fields matters.

continents:
    name: <string>
    regions:
        center_latitude: <int>
        center_longitude: <int>
        width_miles: <int>
        height_miles: <int>
        topography: <string>
        climate: <string>
        biome: <string>
        size: <string>
        name: <string>

IT I

In [133]:
(raw_geography, parsed_geography) = system_completion(geography_prompt())
print(raw_geography)

continents:
  - name: "Auroria"
    regions:
      - center_latitude: -200
        center_longitude: 400
        width_miles: 400
        height_miles: 300
        topography: "mountains"
        climate: "temperate"
        biome: "alpine"
        size: "large"
        name: "The Frozen North"
      - center_latitude: -200
        center_longitude: 0
        width_miles: 300
        height_miles: 200
        topography: "plains"
        climate: "cold"
        biome: "tundra"
        size: "medium"
        name: "The Barren Wasteland"
      - center_latitude: 0
        center_longitude: -250
        width_miles: 200
        height_miles: 300
        topography: "forests"
        climate: "temperate"
        biome: "temperate"
        size: "medium"
        name: "The Elvenwood"
      - center_latitude: -100
        center_longitude: -100
        width_miles: 150
        height_miles: 150
        topography: "hills"
        climate: "temperate"
        biome: "fields"
        size: "sm

In [106]:
parsed_geography

{'continents': [{'location_in_world': 'The Eastern Hemisphere',
   'regions': [{'location_in_continent': 'The Northern Tundra',
     'topography': 'Frozen wasteland of snow and ice, dotted with rocky outcroppings and frozen rivers',
     'climate': 'Arctic and subarctic climate, with extremely cold temperatures and little precipitation',
     'biome': 'Tundra and taiga, with sparse vegetation and occasional conifer forests',
     'size': 'Large',
     'name': 'Frostland'},
    {'location_in_continent': 'The Wild Woodlands',
     'topography': 'Rolling hills and thick forests, with crisp streams and hidden clearings',
     'climate': 'Temperate climate, with mild temperatures and moderate precipitation',
     'biome': 'Deciduous forests and grasslands, with canopy trees and open meadows',
     'size': 'Medium',
     'name': 'Greenhaven'},
    {'location_in_continent': 'The Hills of the Sun',
     'topography': 'Gentle hills and green valleys, crisscrossed with small rivers and dotted wi

In [107]:
geography_continent_context = []

for continent in parsed_geography['continents']:
    name = continent['name']
    location_in_world = continent['location_in_world']
    def get_biome(region): return region['biome']
    biomes = map(get_biome, continent['regions'])
    biomes = ', '.join(biomes)
    line = f"The continent {name} located in the {location_in_world} with {biomes} biomes."
    geography_continent_context.append(line)

geography_continent_context = '\n'.join(geography_continent_context)
print(geography_continent_context)

The continent The Old World located in the The Eastern Hemisphere with Tundra and taiga, with sparse vegetation and occasional conifer forests, Deciduous forests and grasslands, with canopy trees and open meadows, Grasslands and savannas, with scrubby bushes and low trees, Alpine tundra, with sparse vegetation and snow-capped peaks biomes.
The continent The New World located in the The Western Hemisphere with Coral reefs and tropical rainforests, with exotic flora and fauna, Sandy deserts and rocky badlands, with sparse vegetation and nomadic tribes, Grasslands and prairies, with bison and wild horses roaming free, Temperate rainforests and wetlands, with thick underbrush and towering canopies biomes.


In [115]:
def civilizations_prompt(geography_context, civilizations=(4,5), settlements=(3,6)): return f"""
Generate the major fictional fantasy civilizations in the world and their major cities.

There MUST be {range(civilizations)} civilizations in the world.
There MUST be {range(settlements)} major cities per civilization.

The civlizations and settlements should exist in the world's geopgrahy, defined by the below section:

```
{geography_context}
```

The response must be in the following structure.

civilizations:
    location: <reference, name of continent in world geopraphy>
    cities:
        location: <reference, name of region in world geography>
        city_economy_description: <string>
        name: <string>
    ruling_faction: <string>
    name: <string>
"""

civilizations_prompt(raw_geography)

'\nGenerate the major fictional fantasy civilizations in the world and their major cities.\n\nThere MUST be at least 4 and no greater than 5 civilizations in the world.\nThere MUST be at least 3 and no greater than 6 major cities per civilization.\n\nThe civlizations and settlements should exist in the world\'s geopgrahy, defined by the below section:\n\n```\ncontinents:\n  - location_in_world: "The Eastern Hemisphere"\n    regions:\n      - location_in_continent: "The Northern Tundra"\n        topography: "Frozen wasteland of snow and ice, dotted with rocky outcroppings and frozen rivers"\n        climate: "Arctic and subarctic climate, with extremely cold temperatures and little precipitation"\n        biome: "Tundra and taiga, with sparse vegetation and occasional conifer forests"\n        size: "Large"\n        name: "Frostland"\n      - location_in_continent: "The Wild Woodlands"\n        topography: "Rolling hills and thick forests, with crisp streams and hidden clearings"\n     

In [116]:
(raw_civilizations, parsed_civlizations) = system_completion(civilizations_prompt(raw_geography))
print(raw_civilizations)

civilizations:
  - location: "The Old World"
    cities:
      - location: "Frostland"
        city_economy_description: "A bustling trade city, known for trading exotic furs and ice sculptures."
        name: "Frostford"
      - location: "Frostland"
        city_economy_description: "A merchant city, specializing in the trade of gems, precious jewels, and ornate sculptures."
        name: "Crystalbarrow"
      - location: "Frostland"
        city_economy_description: "A massive seaport city, sustaining the trade industry of the northern tundra."
        name: "Seaborn"
      - location: "Greenhaven"
        city_economy_description: "A quiet, secluded wood elf village known for its craftwork and wine industry."
        name: "Serenity Vale"
      - location: "Greenhaven"
        city_economy_description: "A large town, known for lumber, paper, and musical instrument production."
        name: "Harvest Home"
      - location: "Greenhaven"
        city_economy_description: "A fortified

In [118]:
# validation

for civilization in parsed_civlizations['civilizations']:
    name = civilization['name']
    print(f"Civilization: name")
    print(f"\tRuled by: {civilization['ruling_faction']}")
    location_continent = civilization['location']
    matches = [c for c in parsed_geography['continents'] if c['name'] == location_continent]
    if not len(matches) == 1:
        raise Exception(f"for civ {name} couldn't find continent {location_continent}")
    continent = matches[0]
    print(f"\tLocated on the continent {location_continent}")
    for city in civilization['cities']:
        location_region = city['location']
        matches = [r for r in continent['regions'] if r['name'] == location_region]
        if not len(matches) == 1:
            raise Exception(f"for city {city['name']} couldn't find region {location_region} in {location_continent}")
        region = matches[0]
        print(f"\t\tCity: {city['name']} (Region: {region['name']}, {region['location_in_continent']}, {region['topography']}, {region['biome']})")
        print(f"\t\t\t{city['city_economy_description']}")

Civilization: name
	Ruled by: Council of Elders
	Located on the continent The Old World
		City: Frostford (Region: Frostland, The Northern Tundra, Frozen wasteland of snow and ice, dotted with rocky outcroppings and frozen rivers, Tundra and taiga, with sparse vegetation and occasional conifer forests)
			A bustling trade city, known for trading exotic furs and ice sculptures.
		City: Crystalbarrow (Region: Frostland, The Northern Tundra, Frozen wasteland of snow and ice, dotted with rocky outcroppings and frozen rivers, Tundra and taiga, with sparse vegetation and occasional conifer forests)
			A merchant city, specializing in the trade of gems, precious jewels, and ornate sculptures.
		City: Seaborn (Region: Frostland, The Northern Tundra, Frozen wasteland of snow and ice, dotted with rocky outcroppings and frozen rivers, Tundra and taiga, with sparse vegetation and occasional conifer forests)
			A massive seaport city, sustaining the trade industry of the northern tundra.
		City: Se